# Mission to Mars

In [1]:
import requests
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd

## Part 1: Scraping
### NASA Mars News

In [2]:
response = requests.get("https://mars.nasa.gov/news/")

In [3]:
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
else:
    print("There was an error going to your webpage")

In [4]:
print(response.status_code)

200


In [5]:
print(soup.title)

<title>News  – NASA’s Mars Exploration Program </title>


In [6]:
for t1 in soup.findAll('div', {"class": "image_and_description_container"}):
    print(t1.findAll("div",{"class": "rollover_description_inner"})[0].text)


Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.


The rover recently drilled two samples, and both showed the highest levels of clay ever found during the mission.


For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.


Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.


NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opportunity's record-breaking exploration laid the groundwork for future missions to the Red Planet.


In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.



In [7]:
# wrong.. need to delete
content_divs = soup.findAll("div", {"class": "content_title"})
for div in content_divs:
   print(div.findAll("a")[0].text)


NASA Invites Students to Name Mars 2020 Rover


NASA's Curiosity Mars Rover Finds a Clay Cache


Why This Martian Full Moon Looks Like Candy


NASA Garners 7 Webby Award Nominations


NASA's Opportunity Rover Mission on Mars Comes to End


NASA's InSight Places First Instrument on Mars



# Part 2 Get the Featured Image with Splinter

In [8]:
# Chromedriver path
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

# Visit the url for JPL Featured Space Image
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)

In [9]:
soup2 = BeautifulSoup(response.text, 'html.parser')

In [10]:
main_url = "https://www.jpl.nasa.gov"
featured_image = soup2.findAll("article", {"class": "carousel_item"})[0]["style"].split("background-image: url('")[1][:-3]
featured_image_url = main_url + featured_image
featured_image_url

IndexError: list index out of range

# Part 3: Mars Weather Twitter

In [11]:
response = requests.get("https://twitter.com/marswxreport?lang=en")

In [12]:
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
else:
    print("There was an error going to your webpage")

In [13]:
mars_weather = soup.findAll("div", {"class": {"js-tweet-text-container"}})[0].text
print(mars_weather)


Weeeeee!  https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA23461 …pic.twitter.com/iuEqJjsQ4G



# Part 4: Mars Facts

In [14]:
res = requests.get("https://space-facts.com/mars/")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[1] 
df = pd.read_html(str(table))
print(df[0])

                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


# Part 5: Mars Hemisphere

In [24]:
# Chromedriver path
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

# Visit the url for JPL Featured Space Image
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')
results = browser.find_by_css('img[class="thumb"]')
num_tries = len(results)
hemisphere_image_urls = []

for trial in range(0, num_tries):
    browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')
    results = browser.find_by_css('img[class="thumb"]')
    results[trial].click()
    soup = BeautifulSoup(browser.html)
    hemisphere_image_urls.append(dict(title=soup.find_all("h2", {"class": {"title"}})[0].text,
                                     img_url=soup.find_all("a", string="Original")[0]['href'])
                                )
    
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]
